In [1]:
%cd ..

/Users/jason/repos/diffusion-motion-inbetweening


In [2]:
from pathlib import Path

import numpy as np
from convert.joints2bvh.joints2bvh import Joint2BVHConvertor
from tqdm import tqdm

/Users/jason/repos/diffusion-motion-inbetweening/convert/joints2bvh/InverseKinematics.py:2: UserWarning: A NumPy version >=1.22.4 and <2.3.0 is required for this version of SciPy (detected version 1.21.5)
  import scipy.linalg as linalg


# `new_joints` - New Joints
- Shape: (F, J, 3)
- Might contain various skeleton sizes
- Easiest to plot

In [3]:
base_path = Path("dataset/HumanML3D")
data_path = base_path / "new_joints_abs_3d"
output_path = base_path / "bvh"
output_path.mkdir(exist_ok=True)

split_files = {
    "train": base_path / "train.txt",
    "val": base_path / "val.txt",
    "test": base_path / "test.txt"
}


In [4]:
split_ids = {}

for split, path in split_files.items():
    with open(path, "r") as f:
        split_ids[split] = f.read().splitlines()

{split: len(ids) for split, ids in split_ids.items()}

{'train': 23384, 'val': 1460, 'test': 4384}

In [8]:
converter = Joint2BVHConvertor()
n_convert = 32
iterations = 100
splits = ["test"]

for split in splits:
    ids = split_ids[split]

    if split in ["train", "val"]:
        continue

    split_dir = output_path / split
    split_dir.mkdir(exist_ok=True)

    for id in tqdm(ids[:n_convert], desc=f"Converting {split}"):

        # Complex Debug Motion: Walk stairs up and down
        # if id != "002246":
        #     continue

        npy_path = data_path / f"{id}.npy"
        text_path = base_path / "texts" / f"{id}.txt"

        with text_path.open("r") as f:
            first_text = f.readline().split("#")[0]
            first_text = first_text.replace(" ", "_").replace(":", "_").replace(".", "").replace(",", "")

        bvh_path = split_dir / f"{id}_fromjoint{iterations}_{first_text}.bvh"

        joints = np.load(npy_path)
        bvh = converter.convert(joints, bvh_path, iterations=iterations, foot_ik=False)


Converting test: 100%|██████████| 32/32 [00:48<00:00,  1.50s/it]


In [6]:
import ipywidgets
import matplotlib.pyplot as plt

# from humanml/utils/paramUtil.py
t2m_kinematic_chain = [[0, 2, 5, 8, 11], [0, 1, 4, 7, 10], [0, 3, 6, 9, 12, 15], [9, 14, 17, 19, 21], [9, 13, 16, 18, 20]]

@ipywidgets.interact(frame=(0, joints.shape[0]-1))
def plot(frame=0):
    fig = plt.figure(figsize=(9, 9))
    plt.suptitle(f"{id}")

    ax = fig.add_subplot(111, projection='3d')
    ax.set_xlabel('X-axis')
    ax.set_ylabel('Y-axis')
    ax.set_zlabel('Z-axis')

    ax.view_init(elev=10, azim=35, vertical_axis="y")
    ax.axis("equal")

    ax.set_ylim(0, 2)
    ax.set_xlim(-1, 1)
    ax.set_zlim(-1, 1)

    for chain in t2m_kinematic_chain:
        ax.plot3D(
            joints[frame, chain, 0],
            joints[frame, chain, 1],
            joints[frame, chain, 2],
        )

    plt.show()

interactive(children=(IntSlider(value=0, description='frame', max=198), Output()), _dom_classes=('widget-inter…